In [1]:
%load_ext autoreload
%autoreload 2

import torch 
import numpy as np
import pandas as pd

from utils import load_data_and_model, create_dataframe, create_submit_dataframe, save_dataframe

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [8]:
model_paths = {
    'QNLI': '/dfs/scratch0/chami/metal/metal/mmtl/checkpoints/15-2-2019/QNLI_21_28_59/checkpoints/',
    'STSB': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/14-2-2019/STSB_17_59_04/',
    'SST2': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/14-2-2019/SST2_00_01_36/',
    'COLA': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/14-2-2019/COLA_19_48_08/',
    'RTE':'/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/13-2-2019/RTE_22_52_53/',
    'WNLI': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/13-2-2019/WNLI_23_05_55/',
    'QQP': '/dfs/scratch0/vschen/metal-mmtl/logs/checkpoints/15-2-2019/QQP_00_55_59/',
    'MRPC':'/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/13-2-2019/MRPC_23_21_58/',
    'MNLI': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/15-2-2019/MNLI_22_13_38/'
}

for task_name in ['QNLI']:
    print(task_name)
    model_path = model_paths[task_name]
    if model_path is None:
        continue
    
    if task_name is 'MNLI':
        split="test_mismatched"
        model,dl = load_data_and_model(model_path+'best_model.pth',task_name,split)
        df_error = create_submit_dataframe(model_path+'best_model.pth',task_name,model,dl)
        save_dataframe(df_error,task_name+'-mm')
        
        split="test_matched"
        model,dl = load_data_and_model(model_path+'best_model.pth',task_name,split)
        df_error = create_submit_dataframe(model_path+'best_model.pth',task_name,model,dl)
        save_dataframe(df_error,task_name+'-m')
        
    else:
        split = "test"
        model,dl = load_data_and_model(model_path+'best_model.pth',task_name,split)
        df_error = create_submit_dataframe(model_path+'best_model.pth',task_name,model,dl)
        save_dataframe(df_error,task_name)

QNLI


Loading QNLI Dataset


100%|██████████| 171/171 [00:21<00:00,  8.10it/s]


Saved dataframe to:  ./QNLI.tsv


**In case one is too lazy to zip manually**

In [13]:
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.split('.')[-1] == 'tsv':
                ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile('submission.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('./', zipf)
zipf.close()